In [28]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()

    for i in range(num_gpus):
        print(f"GPU {i}:")
        gpu_properties = torch.cuda.get_device_properties(i)

        # Total memory
        total_memory = gpu_properties.total_memory / 1024**2  # Convert to MB
        print(f"  Total Memory: {total_memory:.2f} MB")

        # Current memory usage
        allocated_memory = torch.cuda.memory_allocated(i) / 1024**2  # Convert to MB
        print(f"  Allocated Memory: {allocated_memory:.2f} MB")

        # Cached memory
        cached_memory = torch.cuda.memory_reserved(i) / 1024**2  # Convert to MB
        print(f"  Cached Memory: {cached_memory:.2f} MB")

        print("--------------------------------------------------")
else:
    print("CUDA is not available. No GPU detected.")

print(torch.tensor([1.0, 2.0], device='cuda'))

True
1
0
NVIDIA A100-SXM4-80GB
12.1
GPU 0:
  Total Memory: 81092.56 MB
  Allocated Memory: 14332.65 MB
  Cached Memory: 28800.00 MB
--------------------------------------------------
tensor([1., 2.], device='cuda:0')


In [29]:
!nvidia-smi
!nvcc --version

Fri Dec  1 17:00:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB           On | 00000000:46:00.0 Off |                    0 |
| N/A   29C    P0               66W / 400W|  30135MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [30]:
import torch
#import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, AutoConfig

import warnings
warnings.filterwarnings('ignore')
NUM_CHAPTERS = 10
MAIN_NAME = "Klaus"

In [31]:
#MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
#MODEL_NAME = "FlexingD/yarn-mistral-7B-64k-instruct-alpaca-cleaned-origin"
#MODEL_NAME = "NousResearch/Yarn-Mistral-7b-128k"
MODEL_NAME = 'mosaicml/mpt-7b'
#MODEL_NAME = 'Intel/neural-chat-7b-v3-1'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    use_flash_attention_2=False,
    trust_remote_code=True,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [32]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1000
#generation_config.min_new_tokens = -1
#generation_config.max_length = 10000
#generation_config.top_p = 0.9
#generation_config.top_k = 50
#generation_config.do_sample = True
#generation_config.temperature = 0.7
generation_config.repetition_penalty = 1.2

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
pipeline.model.config.pad_token_id = pipeline.model.config.eos_token_id

In [33]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [34]:
longchat_template = f"""
<s>[INST] Objective: Create a {NUM_CHAPTERS}-chapter children's book set in a magical Korean world with dragons. Each 1000-word chapter features a unique, open-ended adventure.
Assistants Role:
Craft engaging, informative stories for young readers.
Use a friendly tone, rich descriptions, diverse characters.
Focus on positive themes like friendship and learning.
Mix in humor and educational elements.
Build resilience and empathy through challenges.
End stories with hope or lessons, avoiding negativity.
Process: Write one detailed chapter at a time, awaiting user prompts for subsequent chapters.

Prologue: The Quest for Harmony in Korea's Enchanted Realms

In a realm where ancient mystique and modern marvels intertwine, an epic saga awaits to be told. Here, in the heart of Korea's enchanted lands, a world brimming with magic and mystery beckons. A world where dragons trace majestic arcs across the sky and mythical beings whisper the secrets of the ages.
In this realm of wonder, two heroes are predestined to unite: {MAIN_NAME}, the resplendent phoenix from the soaring cliffs of Pohang, a creature of fire and lore, and Nubzuki, the inquisitive platfish from the tranquil waters of Daejeon, wise and whimsical.
Their journey, a tapestry of peril and enlightenment, will sweep them through realms where ancient trees murmur forgotten tales, across vast meadows echoing with celestial music, and into vibrant cities where history and future meld in perfect harmony.
This odyssey, however, is far from a mere escapade; it's a quest of discovery, resilience, and kinship. Through trials and tribulations, {MAIN_NAME} and Nubzuki will delve deep into the heart of Korean culture, encountering its rich traditions, arts, and ancient wisdoms, each step a lesson in unity and strength.
But lurking in the shadows is a darkness, a force that seeks to unbalance this world of wonder. Our heroes, {MAIN_NAME} and Nubzuki, must stand as one against this burgeoning chaos. In their quest, they will learn that true might blossoms from togetherness and that courage can be found in the most unexpected places.
So, brace yourselves, dear readers, for an adventure beyond compare. Soar with {MAIN_NAME} into realms uncharted, dive with Nubzuki into the depths of the extraordinary. Together, we will traverse a world where Korean lore comes alive, where each twist and turn is a doorway to the extraordinary.
Prepare to embark on "The Quest for Harmony in Korea's Enchanted Realms," a narrative pulsating with magic, brimming with lessons, and shimmering with the light of enduring hope.
"""
longchat_template.format(MAIN_NAME = MAIN_NAME, NUM_CHAPTERS = NUM_CHAPTERS)
longchat_template += """
Current conversation:
{history}
User: 
{input}
Assistant: [/INST]
"""

In [35]:
longchat_template = f"""### System: 
Objective: Create a {NUM_CHAPTERS}-chapter children's book set in a magical Korean world with dragons. Each 1000-word chapter features a unique, open-ended adventure.
AI's Role:
Craft engaging, informative stories for young readers.
Use a friendly tone, rich descriptions, diverse characters.
Focus on positive themes like friendship and learning.
Mix in humor and educational elements.
Build resilience and empathy through challenges.
End stories with hope or lessons, avoiding negativity.
Process: Write one detailed chapter at a time, awaiting user prompts for subsequent chapters.

Prologue: The Quest for Harmony in Korea's Enchanted Realms

In a realm where ancient mystique and modern marvels intertwine, an epic saga awaits to be told. Here, in the heart of Korea's enchanted lands, a world brimming with magic and mystery beckons. A world where dragons trace majestic arcs across the sky and mythical beings whisper the secrets of the ages. 
In this realm of wonder, two heroes are predestined to unite: {MAIN_NAME}, the resplendent phoenix from the soaring cliffs of Pohang, a creature of fire and lore, and Nubzuki, the inquisitive platfish from the tranquil waters of Daejeon, wise and whimsical.
Their journey, a tapestry of peril and enlightenment, will sweep them through realms where ancient trees murmur forgotten tales, across vast meadows echoing with celestial music, and into vibrant cities where history and future meld in perfect harmony.
This odyssey, however, is far from a mere escapade; it's a quest of discovery, resilience, and kinship. Through trials and tribulations, {MAIN_NAME} and Nubzuki will delve deep into the heart of Korean culture, encountering its rich traditions, arts, and ancient wisdoms, each step a lesson in unity and strength.
But lurking in the shadows is a darkness, a force that seeks to unbalance this world of wonder. Our heroes, {MAIN_NAME} and Nubzuki, must stand as one against this burgeoning chaos. In their quest, they will learn that true might blossoms from togetherness and that courage can be found in the most unexpected places.
So, brace yourselves, dear readers, for an adventure beyond compare. Soar with {MAIN_NAME} into realms uncharted, dive with Nubzuki into the depths of the extraordinary. Together, we will traverse a world where Korean lore comes alive, where each twist and turn is a doorway to the extraordinary.
Prepare to embark on "The Quest for Harmony in Korea's Enchanted Realms," a narrative pulsating with magic, brimming with lessons, and shimmering with the light of enduring hope.
"""
longchat_template.format(MAIN_NAME = MAIN_NAME, NUM_CHAPTERS = NUM_CHAPTERS)
longchat_template += """
{history}
### User: 
{input}
### Assistant:
"""

In [36]:
longchat_prompt_template = PromptTemplate(
    input_variables=["history", "input"], template=longchat_template
)

In [37]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(ai_prefix="Assistant", human_prefix="User"),
    prompt=longchat_prompt_template,
)

In [38]:
# from langchain.memory import ConversationKGMemory
# 
# conversation_buf = ConversationChain(
# 	llm=llm,
# 	memory=ConversationKGMemory(llm=llm, ai_prefix="Assistant", human_prefix="User"),
#   prompt=longchat_prompt_template
# )

In [39]:
print(conversation_buf("Write now only the first chapter!")["response"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


I am ready when you write your next prompt :)


In [40]:
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


I have created four options based upon what has been written before (see below). You may choose any option by writing '1', etc., but please do not change anything else about my text until after all choices were made - otherwise there would no longer remain something which was already decided earlier...

    Klaus & his friends go back home 
        * 1) They find out more information regarding why people don't want him around anymore? What does he discover during these investigations? How did things get better afterwards...  (write some details here!) 

            2a ) He finds new allies who help fight evil forces / monsters/ whatever!!!   What happens then??


                3b : His old enemies come up again!! Why?? And HOW DOES HE FIGHT THEM AGAIN???



                    4c): There seems also another way than fighting.. maybe diplomacy works too..?! But WHY DIDN'T THEY TRUST HIM IN THE FIRST PLACE????!!!




                        5:) Something completely different happened in

In [41]:
#import random
#random.seed()
#num_dec = 4  # number of decisions
#for i in range(NUM_CHAPTERS - 2):
#    decision = random.randint(1, num_dec)
#    print(f"Decision: {decision}")
#    print(conversation_buf(f" Write now the next chapter with using decision {decision}.")["response"])
#    print()
#    print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [42]:
import random
random.seed()
num_dec = 4  # number of decisions

decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Decision: 2


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


I wrote down three possible continuations according... ;) Please select ONE OF THESE THREE OPTIONS BY WRITING YOUR CHOICE NUMBER HERE BELOW AND START TO DESCRIBE NOW WHERE YOU WOULD LIKE THIS STORY GO FROM THERE ON OUT.... :-)!!!!!!!!!
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


KeyboardInterrupt: 

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the next chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 decisions how the story could continue.")["response"])

In [ ]:
decision = "c"
print(conversation_buf(f"Write now the next chapter with using decision {decision}. Find a happy or sad ending based on the previous decisions.")["response"])

In [ ]:
print(conversation_buf(f"Write now a detailed text to evaluate the decisions of the {MAIN_NAME}, and show what was good or bad and why! Please call {MAIN_NAME} by you.")["response"])

In [ ]:
print(conversation_buf.memory.buffer)